In [1]:
import torch
# from PIL import Image
import os
import pandas as pd
import shutil
import warnings
from tqdm import tqdm
import json
warnings.filterwarnings(action='ignore')


In [2]:
weights_path = '../models/House_best.pt'
image_folder_path = '../../data/noannotated_data/house/'
# save_folder_path = '../detected_img//'
usuable_names = {
    "집전체" : 0,
    '지붕' : 1,
    '문' : 3, 
    '창문' : 4}

In [3]:
model = torch.hub.load('ultralytics/yolov5', 'custom', path=weights_path, force_reload=True)

image_paths = []
for filename in os.listdir(image_folder_path):
    if filename.endswith(".jpg"):
        image_paths.append(image_folder_path+filename)
image_paths.sort()

test_house_df = pd.read_csv('../../data/test_house.csv')
results_house_df = pd.DataFrame(columns=test_house_df.columns)
house_boxes_list = []
house_boxes_listin_list = []

except_images = []
# id,door_yn,loc,roof_yn,window_cnt,size
for index, filename in tqdm(enumerate(image_paths)):
    results = model(filename, size=512)
    df = results.pandas().xywh[0]
    
    file_id = filename.split('/')[-1].split('.')[0]
    
    class_boxes = []
    window_cnt = 0
    df_index = {
        "id" : file_id,
        "door_yn" : "n",
        "roof_yn" : "n"}
    for i in range(len(df)):
        
        box = df.iloc[i, :]
        box_data = {
            'cls': box.loc['class'],
            'conf': box.confidence,
            'xcenter': box.loc['xcenter'],
            'ycenter': box.ycenter,
            'width': box.width,
            'height': box.height,
            'area' : box.width * box.height / (512 * 512),
            'x_center_point' : box.xcenter / 512
        }
        house_boxes_listin_listin_dict = {
            'id': file_id,
            'class': box.loc['class'],
            'confidence': box.confidence,
            'xcenter': box.loc['xcenter'],
            'ycenter': box.ycenter,
            'width': box.width,
            'height': box.height,
        }
        house_boxes_listin_list.append(house_boxes_listin_listin_dict)
        if box_data['cls'] == usuable_names['문']:
            df_index['door_yn'] = "y"
        elif box_data['cls'] == usuable_names['집전체']:
            class_boxes.append(box_data)
        elif box_data['cls'] == usuable_names['지붕']:
            df_index['roof_yn'] = "y"
        elif box_data['cls'] == usuable_names['창문']:
            window_cnt += 1
    house_boxes_list.append(house_boxes_listin_list)
    try:
        class_box = max(class_boxes, key=lambda x: x['area'])
    except:
        except_images.append(file_id)
        df_index["door_yn"] = None
        df_index["roof_yn"] = None
        df_index["size"] = None
        df_index["loc"] = None
        df_index["window_cnt"] = None
        results_house_df.loc[index] = df_index
        continue

    if class_box["area"] >= 0.6:
        df_index["size"] = "big"
    elif class_box["area"] >= 0.16:
        df_index["size"] = "middle"
    else:
        df_index["size"] = "small"

    if class_box["x_center_point"] < 1/3:
        df_index["loc"] = "left"
    elif class_box["x_center_point"] < 2/3:
        df_index["loc"] = "center"
    else:
        df_index["loc"] = "right"

    if window_cnt >= 3:
        df_index["window_cnt"] = "more than 3"
    elif window_cnt >= 1:
        df_index["window_cnt"] = "1 or 2"
    else:
        df_index["window_cnt"] = "absence"
    
    results_house_df.loc[index] = df_index
results_house_df.to_csv('../../data/results_house.csv', index=False)

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /home/connet/.cache/torch/hub/master.zip
YOLOv5 🚀 2024-8-3 Python-3.9.19 torch-2.4.0+cu121 CPU

Fusing layers... 
Model summary: 574 layers, 140105440 parameters, 0 gradients, 208.2 GFLOPs
Adding AutoShape... 
934it [07:41,  2.02it/s]


In [4]:
weights_path = '../models/Tree_best.pt'
image_folder_path = '../../data/noannotated_data/tree/'
# save_folder_path = '../detected_img//'
usuable_names = {
    "나무전체" : 0,
    '수관' : 2,
    '가지' : 3, 
    '뿌리' : 4,
    '열매' : 7}

In [5]:
model = torch.hub.load('ultralytics/yolov5', 'custom', path=weights_path, force_reload=True)

image_paths = []
for filename in os.listdir(image_folder_path):
    if filename.endswith(".jpg"):
        image_paths.append(image_folder_path+filename)
image_paths.sort()

test_tree_df = pd.read_csv('../../data/test_tree.csv')
results_tree_df = pd.DataFrame(columns=test_tree_df.columns)
tree_boxes_dict = {}

except_images = []

for index, filename in tqdm(enumerate(image_paths)):
    results = model(filename, size=512)
    df = results.pandas().xywh[0]
    
    file_id = filename.split('/')[-1].split('.')[0]
    tree_boxes_dict[file_id] = df
    
    class_boxes = []
    window_cnt = 0
    df_index = {
        "id" : file_id,
        "branch_yn" : "n",
        "root_yn" : "n",
        "crown_yn" : "n",
        "fruit_yn" : "n",
        "gnarl_yn" : "n"
        }
    for i in range(len(df)):
        
        box = df.iloc[i, :]
        box_data = {
            'cls': box.loc['class'],
            'conf': box.confidence,
            'xcenter': box.loc['xcenter'],
            'ycenter': box.ycenter,
            'width': box.width,
            'height': box.height,
            'area' : box.width * box.height / (512 * 512),
            'x_center_point' : box.xcenter / 512
        }
        if box_data['cls'] == usuable_names['나무전체']:
            class_boxes.append(box_data)
        elif box_data['cls'] == usuable_names['수관']:
            df_index['crown_yn'] = "y"
        elif box_data['cls'] == usuable_names['가지']:
            df_index['branch_yn'] = "y"
        elif box_data['cls'] == usuable_names['뿌리']:
            df_index["root_yn"] = "y"
        elif box_data['cls'] == usuable_names['열매']:
            df_index["fruit_yn"] = "y"
    try:
        class_box = max(class_boxes, key=lambda x: x['area'])
    except:
        except_images.append(file_id)
        df_index["branch_yn"] = None
        df_index["root_yn"] = None
        df_index["crown_yn"] = None
        df_index["fruit_yn"] = None
        df_index["gnarl_yn"] = None
        df_index["size"] = None
        df_index["loc"] = None
        results_tree_df.loc[index] = df_index
        continue

    if class_box["area"] >= 0.6:
        df_index["size"] = "big"
    elif class_box["area"] >= 0.16:
        df_index["size"] = "middle"
    else:
        df_index["size"] = "small"

    if class_box["x_center_point"] < 1/3:
        df_index["loc"] = "left"
    elif class_box["x_center_point"] < 2/3:
        df_index["loc"] = "center"
    else:
        df_index["loc"] = "right"
    results_tree_df.loc[index] = df_index
results_tree_df.to_csv('../../data/results_tree.csv', index=False)



Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /home/connet/.cache/torch/hub/master.zip
YOLOv5 🚀 2024-8-2 Python-3.9.19 torch-2.4.0+cu121 CPU

Fusing layers... 
Model summary: 574 layers, 140095828 parameters, 0 gradients, 208.2 GFLOPs
Adding AutoShape... 
934it [07:12,  2.16it/s]


In [6]:
weights_path = '../models/PersonF_best.pt'
image_folder_path = '../../data/noannotated_data/person/'
# save_folder_path = '../detected_img//'
usuable_names = {
    "사람전체" : 0,
    '눈' : 3,
    '입' : 5, 
    '팔' : 10,
    '다리' : 12}

In [7]:
model = torch.hub.load('ultralytics/yolov5', 'custom', path=weights_path, force_reload=True)

image_paths = []
for filename in os.listdir(image_folder_path):
    if filename.endswith(".jpg"):
        image_paths.append(image_folder_path+filename)
image_paths.sort()

test_person_df = pd.read_csv('../../data/test_person.csv')
results_person_df = pd.DataFrame(columns=test_person_df.columns)
person_boxes_dict = {}

except_images = []

for index, filename in tqdm(enumerate(image_paths)):
    results = model(filename, size=512)
    df = results.pandas().xywh[0]
    
    file_id = filename.split('/')[-1].split('.')[0]
    person_boxes_dict[file_id] = df
    
    class_boxes = []
    window_cnt = 0
    df_index = {
        "id" : file_id,
        "eye_yn" : "n",
        "leg_yn" : "n",
        "mouth_yn" : "n",
        "arm_yn" : "n"
        }
    for i in range(len(df)):
        
        box = df.iloc[i, :]
        box_data = {
            'cls': box.loc['class'],
            'conf': box.confidence,
            'xcenter': box.loc['xcenter'],
            'ycenter': box.ycenter,
            'width': box.width,
            'height': box.height,
            'area' : box.width * box.height / (512 * 512),
            'x_center_point' : box.xcenter / 512
        }
        if box_data['cls'] == usuable_names['사람전체']:
            class_boxes.append(box_data)
        elif box_data['cls'] == usuable_names['눈']:
            df_index['eye_yn'] = "y"
        elif box_data['cls'] == usuable_names['다리']:
            df_index['leg_yn'] = "y"
        elif box_data['cls'] == usuable_names['입']:
            df_index["mouth_yn"] = "y"
        elif box_data['cls'] == usuable_names['팔']:
            df_index["arm_yn"] = "y"
    try:
        class_box = max(class_boxes, key=lambda x: x['area'])
    except:
        except_images.append(file_id)
        df_index["eye_yn"] = None
        df_index["mouth_yn"] = None
        df_index["leg_yn"] = None
        df_index["arm_yn"] = None
        df_index["size"] = None
        df_index["loc"] = None
        results_person_df.loc[index] = df_index
        continue

    if class_box["area"] >= 0.6:
        df_index["size"] = "big"
    elif class_box["area"] >= 0.16:
        df_index["size"] = "middle"
    else:
        df_index["size"] = "small"

    if class_box["x_center_point"] < 1/3:
        df_index["loc"] = "left"
    elif class_box["x_center_point"] < 2/3:
        df_index["loc"] = "center"
    else:
        df_index["loc"] = "right"
    results_person_df.loc[index] = df_index
results_person_df.to_csv('../../data/results_person.csv', index=False)



Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /home/connet/.cache/torch/hub/master.zip
YOLOv5 🚀 2024-8-2 Python-3.9.19 torch-2.4.0+cu121 CPU

Fusing layers... 
Model summary: 574 layers, 140134276 parameters, 0 gradients, 208.2 GFLOPs
Adding AutoShape... 
934it [07:31,  2.07it/s]


In [17]:
from roboflow import Roboflow

rf = Roboflow(api_key="mwyydJjSc14AA1mjIGl9")
project = rf.workspace().project("house-processing")
project.version(3).deploy(model_type='yolov5', model_path=f'../models')

loading Roboflow workspace...
loading Roboflow project...
View the status of your deployment at: https://app.roboflow.com/jays-workspace/house-processing/3
Share your model with the world at: https://universe.roboflow.com/jays-workspace/house-processing/model/3


In [4]:
with open('../../data/house_v5boxes.json', 'w') as f : 
	json.dump(house_boxes_list, f, indent=4)

TypeError: Object of type int64 is not JSON serializable

In [22]:
tree_boxes_dict
for key in tree_boxes_dict.keys():
    tree_boxes_dict[key] = tree_boxes_dict[key].to_json()
with open('../../data/tree_v5boxes.json', 'w') as f : 
	json.dump(tree_boxes_dict, f, indent=4)

In [23]:
person_boxes_dict
for key in person_boxes_dict.keys():
    person_boxes_dict[key] = person_boxes_dict[key].to_json()
with open('../../data/person_v5boxes.json', 'w') as f : 
	json.dump(person_boxes_dict, f, indent=4)

In [25]:
image_folder_path = '../../data/noannotated_data/house/'
train_house_df = pd.read_csv('../../data/noannotated_data/train_house.csv')
house_none_index = results_house_df[results_house_df['size'].isna()].index.values
none_files = results_house_df.iloc[house_none_index, :1].values
for none_file in none_files:
    shutil.copy(os.path.join(image_folder_path, none_file[0]+'.jpg'), '../../data/yolov5_none_data/house/')


In [73]:
image_house_folder_path = '../../data/noannotated_data/house/'
house_image_list = [x.split(".")[0] for x in os.listdir(image_house_folder_path)]
comp_train_house_df = train_house_df[train_house_df.id.apply(lambda x: x in house_image_list)]
house_notna_index = results_house_df[results_house_df['size'].notna()].index.values
yolo5_comp = results_house_df.iloc[house_notna_index, :]
ori_comp = comp_train_house_df.iloc[house_notna_index, :]

yolo5_comp.reset_index(drop=True, inplace=True)
ori_comp.reset_index(drop=True, inplace=True)
comp_df = yolo5_comp == ori_comp

True_index = []
for i in range(len(comp_df)):
    True_cnt = comp_df.loc[i, :].sum()
    if True_cnt == 6:
        True_index.append(i)
True_index

[10,
 12,
 14,
 17,
 18,
 19,
 20,
 21,
 22,
 28,
 29,
 30,
 37,
 38,
 42,
 43,
 45,
 47,
 49,
 50,
 51,
 57,
 58,
 62,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 74,
 75,
 76,
 78,
 80,
 82,
 84,
 85,
 86,
 87,
 88,
 92,
 94,
 98,
 99,
 105,
 110,
 112,
 113,
 116,
 122,
 124,
 130,
 137,
 138,
 143,
 144,
 146,
 150,
 152,
 158,
 159,
 161,
 162,
 164,
 165,
 168,
 180,
 187,
 192,
 205,
 209,
 213,
 219,
 223,
 224,
 225,
 228,
 232,
 236,
 240,
 241,
 242,
 263,
 265,
 280,
 284,
 285,
 292,
 302,
 306,
 307,
 319,
 324,
 326,
 327,
 332,
 333,
 334,
 335,
 336,
 337,
 339,
 344,
 345,
 348,
 351,
 352,
 356,
 360,
 361,
 362,
 363,
 369,
 373,
 374,
 379,
 388,
 394,
 395,
 398,
 399,
 401,
 405,
 406,
 413,
 414,
 416,
 419,
 420,
 422,
 424,
 427,
 430,
 432,
 433,
 434,
 436,
 438,
 442,
 443,
 445,
 447,
 448,
 450,
 451,
 452,
 453,
 455,
 459,
 460,
 462,
 463,
 464,
 466,
 469,
 471,
 479,
 480,
 488,
 490,
 498,
 505,
 512,
 514,
 517,
 522,
 524,
 525,
 527,
 529,
 531,
 53

In [65]:
print(comp_df.loc[0,:])
for index in comp_df.loc[0,:]:
    print(index)

id             True
door_yn       False
loc           False
roof_yn        True
window_cnt     True
size           True
Name: 0, dtype: bool
True
False
False
True
True
True


In [29]:
image_folder_path = '../../data/noannotated_data/tree/'
train_tree_df = pd.read_csv('../../data/noannotated_data/train_tree.csv')
tree_none_index = results_tree_df[results_tree_df['size'].isna()].index.values
none_files = results_tree_df.iloc[tree_none_index, :1].values
for none_file in none_files:
    shutil.copy(os.path.join(image_folder_path, none_file[0]+'.jpg'), '../../data/yolov5_none_data/tree/')

In [30]:
image_folder_path = '../../data/noannotated_data/person/'
train_person_df = pd.read_csv('../../data/noannotated_data/train_person.csv')
person_none_index = results_person_df[results_person_df['size'].isna()].index.values
none_files = results_person_df.iloc[person_none_index, :1].values
for none_file in none_files:
    shutil.copy(os.path.join(image_folder_path, none_file[0]+'.jpg'), '../../data/yolov5_none_data/person/')